In [1]:
import pandas as pd
import nltk
# nltk.download('all')
import re
import numpy as np
import collections
import sklearn as sk
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [2]:
df = pd.read_csv("../B_Data_Cleaning/cleaned_real_reviews.csv")
df["Review"] = df["Review"].astype("str")
df

,Review,Recommended or Not Recommended,Date Timestamp Created
0,day someone came ps version game quite journey...,True,1645046263
1,replay immediately demolish adam smasher,True,1663224196
2,patch fixed everything quests make sense fixer...,True,1645267750
3,watches edgerunners downloads cyberpunk time life,True,1667117035
4,remember hearing cyberpunk around announced in...,True,1664423074
...,...,...,...
16455,boobs d,True,1608792512
16456,great main story cyberpunk atmosphere good vis...,True,1608792267
16457,got hardware run game definitely,True,1608777643
16458,glitches experience enough hold awesomeness ga...,True,1608777582


In [3]:
# make a set_column to count the number of words in each review
df["set_column"] = df["Review"].apply(lambda x: set(x.split()))
df

,Review,Recommended or Not Recommended,Date Timestamp Created,set_column
0,day someone came ps version game quite journey...,True,1645046263,"{blazing, rebalance, swingin, really, early, a..."
1,replay immediately demolish adam smasher,True,1663224196,"{smasher, replay, demolish, adam, immediately}"
2,patch fixed everything quests make sense fixer...,True,1645267750,"{facial, rolled, want, never, lot, city, way, ..."
3,watches edgerunners downloads cyberpunk time life,True,1667117035,"{watches, cyberpunk, life, edgerunners, downlo..."
4,remember hearing cyberpunk around announced in...,True,1664423074,"{pay, auto, advertisement, thoughts, stop, wat..."
...,...,...,...,...
16455,boobs d,True,1608792512,"{boobs, d}"
16456,great main story cyberpunk atmosphere good vis...,True,1608792267,"{good, atmosphere, definitely, dialog, options..."
16457,got hardware run game definitely,True,1608777643,"{definitely, hardware, got, game, run}"
16458,glitches experience enough hold awesomeness ga...,True,1608777582,"{experience, awesomeness, enough, thisll, patc..."


In [4]:
# keep only 5250 reviews for each based on 'Recommended or Not Recommended" column
temp_recom = df.loc[df["Recommended or Not Recommended"] == True,:]
temp_recom = temp_recom.sample(5250).reset_index(drop=True)

temp_not_recom = df.loc[df["Recommended or Not Recommended"] == False,:]
temp_not_recom = temp_not_recom.sample(5250).reset_index(drop=True)

# combine temp_recom and temp_not_recom
temp = pd.concat([temp_recom, temp_not_recom], axis=0).reset_index(drop=True)

# split the data into train and test
train, test = sk.model_selection.train_test_split(temp, test_size=0.33, random_state=42)

# # split df_recom into train and test
# df_recom = temp_recom.sample(frac=0.67, random_state=42).reset_index(drop=True)
# test_recom = temp_recom.drop(df_recom.index).reset_index(drop=True)
# print(f"train size df_recom: {len(df_recom)}, test size df_recom: {len(test_recom)}")

# # split df_not_recom into train and test
# df_not_recom = temp_not_recom.sample(frac=0.33, random_state=42).reset_index(drop=True)
# test_not_recom = temp_not_recom.drop(df_not_recom.index).reset_index(drop=True)
# print(f"train size df_not_recom: {len(df_not_recom)}, test size df_not_recom: {len(test_not_recom)}")

# # combine test_recom and test_not_recom
# test = pd.concat([test_recom, test_not_recom], axis=0).reset_index(drop=True)

In [5]:
# Dividing dataset into recommended and not recommended
df_recom = train.loc[train["Recommended or Not Recommended"] == True,:]
df_recom = df_recom.reset_index(drop=True)

df_not_recom = train.loc[train["Recommended or Not Recommended"] == False,:]
df_not_recom = df_not_recom.reset_index(drop=True)

In [6]:
# This is to make the set dictionary
# count how many times each word appears in the reviews
bow_recom_temp = collections.Counter([y for x in df_recom.set_column for y in x])
bow_not_recom_temp = collections.Counter([y for x in df_not_recom.set_column for y in x])

bow_recom_set = dict(bow_recom_temp)
bow_not_recom_set = dict(bow_not_recom_temp)

for key in bow_recom_set:
    if key not in bow_not_recom_set:
        bow_not_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_set:
    if key not in bow_recom_set:
        bow_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_recom_set:
    bow_recom_set[key] += 1

for key in bow_not_recom_set:
    bow_not_recom_set[key] += 1


In [7]:
# This is to make the list dictionary
# count how many times each word appears in the reviews
bow_recom = collections.Counter([y for x in df_recom.Review for y in x.split()])
bow_not_recom = collections.Counter([y for x in df_not_recom.Review for y in x.split()])

bow_recom_dict = dict(bow_recom)
bow_not_recom_dict = dict(bow_not_recom)

for key in bow_recom_dict:
    if key not in bow_not_recom_dict:
        bow_not_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_dict:
    if key not in bow_recom_dict:
        bow_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_recom_dict:
    bow_recom_dict[key] += 1

for key in bow_not_recom_dict:
    bow_not_recom_dict[key] += 1
    

In [8]:
bow_recom_sum_vals = sum(bow_recom_dict.values())
bow_not_recom_sum_vals = sum(bow_not_recom_dict.values())

for i in bow_recom_dict:

    bow_recom_dict[i] /= bow_recom_sum_vals

for i in bow_not_recom_dict:

    bow_not_recom_dict[i] /= bow_not_recom_sum_vals

In [9]:
assert len(bow_recom_dict) == len(bow_not_recom_dict)
assert len(bow_recom_set) == len(bow_not_recom_set)

In [10]:
test.drop(columns=["set_column"], inplace=True)

In [11]:
def define_sentiment(element, test_all=False, test_only_tf_abs=False, test_only_idf_abs=False, test_only_prob=False, test_all_abs=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                if test_all:
                    score *= prob_word_given_class*tf*idf
                elif test_only_tf_abs:
                    score *= prob_word_given_class*tf
                    score = abs(score)
                elif test_only_idf_abs:
                    score *= prob_word_given_class*idf
                    score = abs(score)
                elif test_only_prob:
                    score *= prob_word_given_class
                elif test_all_abs:
                    score *= prob_word_given_class*tf*idf
                    score = abs(score)

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    # print(flag_count,i)
                    flag_count += 1
                    # return 'Flag'


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [12]:
test["score_all"] = test.Review.apply(define_sentiment, test_all=True)
test["score_tf"] = test.Review.apply(define_sentiment, test_only_tf_abs=True)
test["score_idf"] = test.Review.apply(define_sentiment, test_only_idf_abs=True)
test["score_freq"] = test.Review.apply(define_sentiment, test_only_prob=True)
test["score_mod_all"] = test.Review.apply(define_sentiment, test_all_abs=True)

In [13]:
print("This is the result with the word 'cyberpunk' included in the corpus:")
print(f"This is our accuracy for (balanced) real data with all the formula: {(sum(test['Recommended or Not Recommended'] == test['score_all'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) real data with only class probability, frequency, and absolute value of TF: {(sum(test['Recommended or Not Recommended'] == test['score_tf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) real data with only class probability, frequency, and absolute value of IDF: {(sum(test['Recommended or Not Recommended'] == test['score_idf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) real data with only class probability and frequency: {(sum(test['Recommended or Not Recommended'] == test['score_freq'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) real data with all the formula, but with the absolute value of the score: {(sum(test['Recommended or Not Recommended'] == test['score_mod_all'])/test.shape[0])*100:.2f}%")

This is the result with the word 'cyberpunk' included in the corpus:
This is our accuracy for (balanced) real data with all the formula: 52.64%
This is our accuracy for (balanced) real data with only class probability, frequency, and absolute value of TF: 76.45%
This is our accuracy for (balanced) real data with only class probability, frequency, and absolute value of IDF: 70.22%
This is our accuracy for (balanced) real data with only class probability and frequency: 74.95%
This is our accuracy for (balanced) real data with all the formula, but with the absolute value of the score: 69.12%


In [14]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_all'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_all']))


              precision    recall  f1-score   support

       False       0.51      0.61      0.56      1699
        True       0.54      0.45      0.49      1766

    accuracy                           0.53      3465
   macro avg       0.53      0.53      0.52      3465
weighted avg       0.53      0.53      0.52      3465



In [15]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_tf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_tf']))


              precision    recall  f1-score   support

       False       0.72      0.86      0.78      1699
        True       0.83      0.67      0.74      1766

    accuracy                           0.76      3465
   macro avg       0.78      0.77      0.76      3465
weighted avg       0.78      0.76      0.76      3465



In [16]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_idf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_idf']))


              precision    recall  f1-score   support

       False       0.72      0.64      0.68      1699
        True       0.69      0.76      0.72      1766

    accuracy                           0.70      3465
   macro avg       0.70      0.70      0.70      3465
weighted avg       0.70      0.70      0.70      3465



In [17]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_freq'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_freq']))


              precision    recall  f1-score   support

       False       0.70      0.87      0.77      1699
        True       0.83      0.64      0.72      1766

    accuracy                           0.75      3465
   macro avg       0.76      0.75      0.75      3465
weighted avg       0.77      0.75      0.75      3465

